In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
train_dataset = torchvision.datasets.CIFAR10(
    train = True,
    root = ".",
    transform = transforms.ToTensor(),
    download = True
)

170499072it [03:02, 935548.93it/s]                                


Extracting ./cifar-10-python.tar.gz to .


In [3]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [4]:
train_dataset.targets

[6,
 9,
 9,
 4,
 1,
 1,
 2,
 7,
 8,
 3,
 4,
 7,
 7,
 2,
 9,
 9,
 9,
 3,
 2,
 6,
 4,
 3,
 6,
 6,
 2,
 6,
 3,
 5,
 4,
 0,
 0,
 9,
 1,
 3,
 4,
 0,
 3,
 7,
 3,
 3,
 5,
 2,
 2,
 7,
 1,
 1,
 1,
 2,
 2,
 0,
 9,
 5,
 7,
 9,
 2,
 2,
 5,
 2,
 4,
 3,
 1,
 1,
 8,
 2,
 1,
 1,
 4,
 9,
 7,
 8,
 5,
 9,
 6,
 7,
 3,
 1,
 9,
 0,
 3,
 1,
 3,
 5,
 4,
 5,
 7,
 7,
 4,
 7,
 9,
 4,
 2,
 3,
 8,
 0,
 1,
 6,
 1,
 1,
 4,
 1,
 8,
 3,
 9,
 6,
 6,
 1,
 8,
 5,
 2,
 9,
 9,
 8,
 1,
 7,
 7,
 0,
 0,
 6,
 9,
 1,
 2,
 2,
 9,
 2,
 6,
 6,
 1,
 9,
 5,
 0,
 4,
 7,
 6,
 7,
 1,
 8,
 1,
 1,
 2,
 8,
 1,
 3,
 3,
 6,
 2,
 4,
 9,
 9,
 5,
 4,
 3,
 6,
 7,
 4,
 6,
 8,
 5,
 5,
 4,
 3,
 1,
 8,
 4,
 7,
 6,
 0,
 9,
 5,
 1,
 3,
 8,
 2,
 7,
 5,
 3,
 4,
 1,
 5,
 7,
 0,
 4,
 7,
 5,
 5,
 1,
 0,
 9,
 6,
 9,
 0,
 8,
 7,
 8,
 8,
 2,
 5,
 2,
 3,
 5,
 0,
 6,
 1,
 9,
 3,
 6,
 9,
 1,
 3,
 9,
 6,
 6,
 7,
 1,
 0,
 9,
 5,
 8,
 5,
 2,
 9,
 0,
 8,
 8,
 0,
 6,
 9,
 1,
 1,
 6,
 3,
 7,
 6,
 6,
 0,
 6,
 6,
 1,
 7,
 1,
 5,
 8,
 3,
 6,
 6,
 8,
 6,
 8,
 4,
 6,
 6,


In [5]:
test_dataset = torchvision.datasets.CIFAR10(
    train = False,
    root = ".",
    transform = transforms.ToTensor(),
    download = True
)

Files already downloaded and verified


In [6]:
K = len(set(train_dataset.targets))
K #set makes a set of distict elements from the array

10

In [7]:
# class CNN(nn.Module): #inherits module
#   def __init__(self,K):  #this is a constructor
#     super(CNN,self).__init__()  #calling parent class constructor, ##CNN is subclass, and self is an object of the subclass
#     self.conv_layers = nn.Sequential(
#         nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=2),
#         nn.ReLU(),
#         nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2),
#         nn.ReLU(),
#         nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=2),
#         nn.ReLU(),
#     )
#     self.deep_layers = nn.Sequential(
#       #nn.Dropout(0.2),
#       nn.Linear(128*3*3,512),
#       #nn.Dropout(0.2),
#       nn.Linear(512,K),
#     )
#   def forward(self,X):
#     out = self.conv_layers(X)
#     out = out.view(out.size(0),-1) #out.size is number of dimentions, -1 is multiplication of everything
#     out = self.deep_layers(out)
#     return out


In [8]:
class CNN(nn.Module): #inherits module
  def __init__(self,K):  #this is a constructor
    super(CNN,self).__init__()  #calling parent class constructor, ##CNN is subclass, and self is an object of the subclass
    self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=2)
    self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2)
    self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=2)
    self.fc1 = nn.Linear(128*3*3,1024)
    self.fc2 = nn.Linear(1024,128)
    self.fc3 = nn.Linear(128,K)
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.relu(self.conv3(x))
    x = x.view(-1, 128 * 3 * 3)
    x = F.dropout(x, p=0.2)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, p=0.3)
    x = self.fc2(x)
    x = F.dropout(x, p=0.4)    
    x = self.fc3(x)
    return x


In [9]:
model = CNN(K)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cpu


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=1152, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [12]:
batch = 32
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    batch_size = batch,
    shuffle = False
)

In [13]:
def minibatch(model,criterion,optimizer,train_loader,test_loader,epochs):
  i=0
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    train_loss = []
    for inputs,targets in train_loader:
      inputs = inputs.to(device)
      targets = targets.to(device)

      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterion(outputs,targets)

      train_loss.append(loss.item())

      loss.backward()
      optimizer.step()
    
    train_loss = np.mean(train_loss)
    train_losses[it] = train_loss

    test_loss = []
    for inputs,targets in test_loader:
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = criterion(outputs,targets)

      test_loss.append(loss.item())

    test_loss = np.mean(test_loss)
    test_losses[it] = test_loss
    print('Epoch',it+1,'/',epochs,'Train Loss:',train_loss,'Test Loss:',test_loss)

  return test_losses,train_losses

In [14]:
test_loss,train_loss = minibatch(model,criterion,optimizer,train_loader,test_loader,20)

In [ ]:
plt.plot(train_loss, label='train_loss')
plt.plot(test_loss, label='test_loss')
plt.legend()
plt.show()

In [ ]:
x_test = test_dataset.data
y_test = np.array(test_dataset.targets)
p_test = np.array([])

n_correct = 0.
n_total = 0.
for inputs, targets in train_loader:
  # move data to GPU
  inputs, targets = inputs.to(device), targets.to(device)
  # Forward pass
  outputs = model(inputs)

  # Get prediction
  # torch.max returns both max and argmax
  _, predictions = torch.max(outputs, 1)
  
  # update counts
  n_correct += (predictions == targets).sum().item()
  n_total += targets.shape[0]

train_acc = n_correct / n_total


n_correct = 0.
n_total = 0.
for inputs, targets in test_loader:
  # move data to GPU
  inputs, targets = inputs.to(device), targets.to(device)
  
  # Forward pass
  outputs = model(inputs)

  # Get prediction
  # torch.max returns both max and argmax
  _, predictions = torch.max(outputs, 1)
  p_test = np.concatenate((p_test, predictions.cpu().numpy()))

  
  # update counts
  n_correct += (predictions == targets).sum().item()
  n_total += targets.shape[0]

test_acc = n_correct / n_total
print(f"Train acc: {train_acc:.4f}, Test acc: {test_acc:.4f}")

In [ ]:
(p_test == y_test).mean()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_test,p_test)

In [ ]:
# label mapping
labels = '''airplane
automobile
bird
cat
deer
dog
frog
horse
ship
truck'''.split()

In [ ]:
p_test = p_test.astype('int64')

In [ ]:
misclassified_idx = np.where(p_test != y_test) #returns a weird tupple, we need nparray of tupple
misclassified_idx = misclassified_idx[0]
#chose any random image index
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i].reshape(32,32,3))
plt.title("True label: %s Predicted: %s" % (labels[y_test[i]], labels[p_test[i]]));